# Create the molecule and FCI dump with pyscf

In [ ]:
import pyscf
import pyscf.cc
import pyscf.mcscf
import ffsim
import numpy as np
import matplotlib.pyplot as plt
from pyscf.tools import fcidump

open_shell = False
spin_sq = 0

atom = lambda R: [["H", (0, 0, 0)], ["H", (0, 0, R)]]
R = 0.75  

mol = pyscf.gto.Mole()
mol.build(atom=atom(R),basis="sto-3g",unit="Angstrom")
 
n_frozen = 0
active_space = range(n_frozen, mol.nao_nr())
 
# Get molecular integrals
scf = pyscf.scf.RHF(mol).run()
norb = len(active_space)
n_electrons = int(sum(scf.mo_occ[active_space]))
num_elec_a = (n_electrons + mol.spin) // 2
num_elec_b = (n_electrons - mol.spin) // 2
cas = pyscf.mcscf.CASCI(scf, norb, (num_elec_a, num_elec_b))
mo = cas.sort_mo(active_space, base=0)
hcore, nuclear_repulsion_energy = cas.get_h1cas(mo)
mol = pyscf.M(
    atom=f"H 0 0 0; H 0 0 {R}",
    basis="sto-3g",
    unit="Angstrom",
)
mf = mol.RHF().run()

fcidump.from_scf(mf, "fcidump.txt")

# Load your IBM job bitstring samples 

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum_platform',
    instance='<insert_your_instance_name_here>'
)
job = service.job('<insert_your_job_id_here>')
primitive_result = job.result()
pub_result = primitive_result[0]
bitstring_samples = pub_result.data.meas

In [6]:
bitstring_samples.get_bitstrings()

['0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0011',
 '0100',
 '0111',
 '0111',
 '0010',
 '0101',
 '0101',
 '0101',
 '1001',
 '0101',
 '0101',
 '0101',
 '0101',
 '0001',
 '0001',
 '0101',
 '0101',
 '0101',
 '0100',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0111',
 '0111',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '1101',
 '0101',
 '0111',
 '0101',
 '0011',
 '0101',
 '0001',
 '0101',
 '0101',
 '0110',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0100',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0100',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0100',
 '0101',
 '0111',
 '1001',
 '1101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0101',
 '0111',
 '0111',
 '0101',
 '0101',
 '0101',
 '0101',
 

# Run it through SQD-SBD python interface

In [4]:
import sqd_hpc_py

counts = pub_result.data.meas.get_counts()
total = sum(counts.values())
bitstrings = list(counts.keys())
weights = [c / total for c in counts.values()]

res = sqd_hpc_py.run_sqd_sbd(
    bitstrings,
    weights,
    num_elec_a,
    num_elec_b,
    fcidump_path="fcidump.txt",
)

 Elapsed time for helper construction 0.018017 (sec) 
 Elapsed time for init 1.7e-05 (sec) 
 Davidson iteration 0.0 (tol=0.181772): -1.11615
 Davidson iteration 0.1 (tol=2.22045e-16): -1.13712 0.459805
 Elapsed time for davidson 0.070644 (sec) 
 Elapsed time for diagonalization 0.070664 (sec) 
 Elapsed time for mult 0.000175 (sec) 
 Energy = -1.137117067345731
 Elapsed time for measurement 1.6e-05 (sec) 
